### https://github.com/belkanov

In [ ]:
"""
1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и
реализовать функцию, которая будет добавлять только новые вакансии в вашу базу.
2. Написать функцию, которая производит поиск и выводит на экран вакансии с
заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).
Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов
с рейтингом не ниже введенного или качеством не ниже введенного
(то есть цифра вводится одна, а запрос проверяет оба поля)
"""
import logging
from pprint import pprint
from time import sleep
from typing import Optional

import requests
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
from pymongo.collection import Collection
from pymongo.errors import DuplicateKeyError
from bson.objectid import ObjectId

from constants import *

logging.basicConfig(format='%(asctime)s | %(levelname)-8s | %(name)s | %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger('job_scraper')
logger.setLevel(logging.INFO)

# если включить - можно увидеть редиректы
# requests_log = logging.getLogger("urllib3")
# requests_log.setLevel(logging.DEBUG)
# requests_log.propagate = True


def get_response(url, headers, params=None):
    timeouts = (5, 5)  # conn, read

    for i in range(5):
        response = requests.get(url,
                                headers=headers,
                                params=params,
                                timeout=timeouts)
        if response.ok:
            logger.debug('response - OK')
            break
        else:
            logger.debug('response - NOT OK (%s)', response.status_code)
            sleep_time = i + 1
            logger.warning(f'Не смог получить ответ для %s. Подождем %d сек.',
                           response.url,
                           sleep_time)
            sleep(sleep_time)
    else:
        raise SystemExit(1, f'Так и не смог получить ответ для {response.url}')

    # на случай редиректа hh.ru -> rostov.hh.ru
    splitted_response_url = response.url.split('/')
    new_main_url = f'{splitted_response_url[0]}//{splitted_response_url[2]}'
    return response, new_main_url


def get_int(re_match):
    return int(re_match.group().replace(' ', ''))


def get_salary(tag):
    if tag is None:
        return Salary(None, None, None)

    text = clear_tag_text(tag)
    # получалось неплохо через всякие сплиты/слайсы/джоины,
    # но потом пришли 'бел. руб.' и все сломалось =)
    # поэтому регулярки
    if 'от' in text:
        re_salary = RE_SALARY.search(text)
        salary = get_int(re_salary)
        re_currency = RE_CURRENCY.search(text, re_salary.end())
        return Salary(salary, None, re_currency.group())
    elif 'до' in text:
        re_salary = RE_SALARY.search(text)
        salary = get_int(re_salary)
        re_currency = RE_CURRENCY.search(text, re_salary.end())
        return Salary(None, salary, re_currency.group())
    else:
        re_min_salary = RE_SALARY.search(text)
        min_salary = get_int(re_min_salary)
        re_max_salary = RE_SALARY.search(text, re_min_salary.end())
        max_salary = get_int(re_max_salary)
        re_currency = RE_CURRENCY.search(text, re_max_salary.end())
        return Salary(min_salary, max_salary, re_currency.group())


def clear_tag_text(tag):
    text = tag.getText()
    text = text.replace('\n', '')
    # внезапно вылезло много пробелов
    splitted = [word for word in text.split() if word]
    text = ' '.join(splitted)
    return text


def parse_vacancy_link(tag):
    link = tag.get('href')
    return f'{MAIN_URL}{link}'


def save_to_file(data, file_name):
    with open(file_name, 'w', encoding='utf8') as f:
        f.write(data)


def parse_response(response, main_url):
    vacancies_info = []

    soup = bs(response.text, 'html.parser')
    anchor = soup.find('div', {'class': 'vacancy-serp-content'})

    vacancy_results = anchor.find('div', {'data-qa': 'vacancy-serp__results'})
    vacancies = vacancy_results.find_all('div', {'class': 'vacancy-serp-item'})
    logger.info('Нашел %d вакансий. Обрабатываю...', len(vacancies))
    for vacancy in vacancies:
        title_tag = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
        salary_tag = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})

        vacancy_info = {
            'name': clear_tag_text(title_tag),
            'salary': get_salary(salary_tag),
            'link': title_tag.get('href'),
            'site': main_url,
        }
        vacancies_info.append(vacancy_info)

    return vacancies_info, anchor


def get_mongo_collection(collection_name: str) -> Optional[Collection]:
    client = MongoClient(host=MONGODB_HOST,
                         port=MONGODB_PORT,
                         username='gb_mongo_root',
                         password='gb_mongo_root_pass')
    db = client[MONGODB_DB_NAME]

    collection = getattr(db, collection_name, None)
    if collection is None:
        raise ValueError(1, f"Коллекция {collection_name} не найдена")

    return collection

# ***************************************************************************
# Комментарий преподавателя по функции ниже:

# Придя на сайт хед хантер видим в названии ссылки присутствует некое числовое значение - 54960009
# Это айдишник который даётся в объявлении с вакансией. 
# Уникальный у каждого объявления. Хед хантер управляет объектами с помощью айдишников
# Это значение можно брать в качестве своего значения ай ди для документа
# _id = ObjectId(f'{vacancy_id:0>24}')
# В итоге 2 одинаковые вакансии к нам в базу не попадут. 
# Объект значения ай ди в монго может быть любого типа. кроме словарей. 
# Здесь к этому объекту приводят ObjectId(f'{vacancy_id:0>24}')

# Для создания уникального поля ай ди у каждой вакансии уникальная ссылка
# Можно взять эту ссылку и прогнать через функцию из библиотеки hashlib
# она формирует последовательности для поданных данных. 
# И для одинаковых данных она сформирует одинаковые последовательности
# можно ссылку в хэш перевести и этот хэш взять уникальным айдишником.

# Монго проверяет на дубли 
# except DuplicateKeyError as e:

# ***************************************************************************






def save_to_mongo(data):
    hh_vacancies = get_mongo_collection('hh_vacancies')
    # вероятно это не самый эффективный вариант вставки большого кол-ва данных
    # (по аналогии с одиночной вставкой в обычном SQL)
    # наверное, лучше бы было обогатить ИДшниками data и вставлять через insert_many...
    # хз - надо гуглить
    # но так проще работать с уже существующими вакансиями
    for row in data:
        # скорее всего ИД уникален только в рамках города (домена) - надо тестить
        # пока норм. всегда можно переделать =)
        try:
            # https://city.hh.ru/vacancy/54960009?fro...
            # -> 54960009
            vacancy_id = row['link'].split('/')[4].split('?')[0]
            # mongo любит только 24-знаковые ИД
            # может это и не best practice, зато не надо новый индекс делать =)
            _id = ObjectId(f'{vacancy_id:0>24}')
            hh_vacancies.insert_one({'_id': _id, **row})
        except DuplicateKeyError as e:
            logger.debug('Для ID=%s уже есть запись. Пропускаем', vacancy_id)  # noqa
            pass

        
        
# ***************************************************************************
# Комментарий преподавателя по функции ниже:

# Оператор '$elemMatch': Он берёт значение 'salary' и сравнивает 
# со всеми значениями salary_value - списка состоящего из зарплат. 
# и если '$elemMatch' пропусая значение 'salary' по всему списку 'salary_value'
# находит элементы большие или равные '$gte'
# данный документ вернётся в качестве результата поиска
# 
# ***************************************************************************

def filter_vacancies_by_salary(salary_value):
    sleep(1)  # дадим логам отлежаться, а то может получиться каша при выводе
    print(f'--- Вакансии с ЗП {salary_value:,}+')
    hh_vacancies = get_mongo_collection('hh_vacancies')
    # запросы конечно смотрятся страшно на фоне обычных SQL =)
    for row in hh_vacancies.find({
        # либо у нас есть значение больше искомого
        '$or': [{'salary': {'$elemMatch': {'$gte': salary_value}}},
                {
                    # либо у нас указана только начальная зарплата
                    '$and': [
                        {'salary.0': {'$ne': None}},
                        {'salary.1': None},
                    ]
                }
                ]
    }):
        pprint(row)


def main():
    page_cnt = 1
    url = VACANCY_URL
    headers = HEADERS
    params = PARAMS
    while True:
        logger.info('Parse page #%d', page_cnt)
        response, main_url = get_response(url, headers=headers, params=params)
        if not response:
            logger.error('NO response from %s', url)
            raise SystemExit(1)

        try:
            vacancies_info, anchor = parse_response(response, main_url)
        except ValueError as e:
            logger.exception(e)
            save_to_file(response.text, 'error_response.html')
            raise SystemExit(1)

        logger.info('Сохраняю их в Mongo')
        save_to_mongo(vacancies_info)

        logger.info('Ищу следующую страницу...')
        next_link = anchor.find('a', {'data-qa': 'pager-next'})
        if next_link:
            logger.info('Нашел.')
            url = f'{main_url}{next_link.get("href")}'
            params = None
            page_cnt += 1
            sleep(1)  # не будем спамить запросами
        else:
            logger.info('Видимо это последняя =) Всего обработано %d страниц',
                        page_cnt)
            break

    filter_vacancies_by_salary(100_000)


if __name__ == '__main__':
    logger.info('--- START')
    main()
    logger.info('--- END')

### https://github.com/Androkotey

In [ ]:
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
from contextlib import contextmanager

# мои модули
from parser_hh import get_vacancies
from mongo_queries import salary_filter


# Будем хранить ссылки на вакансии в множестве
PRIMARY_KEYS = set()
# заменить проверку на
# if collection.find_one('link', doc['link']):
#     print('Документ существует в базе')
# Но лучше сделать update_one с параметром upsert=True


def pprint_cursor_object(cursor):
    for doc in cursor:
        pprint(doc)


@contextmanager
def connect_to_mongodb_collection(database_name, collection_name, delete=False):
    """ Открывает соединение, удаляет коллекцию после отработки, если необходимо, и закрывает соединение """

    client = MongoClient('127.0.0.1', 27017)
    db = client[database_name]
    collection = db[collection_name]

    yield collection

    count_documents = collection.count_documents({})
    if delete:
        db.drop_collection(collection_name)
        print(f"Удалена коллеция с {count_documents} документами")
    client.close()


def add_data_to_collection(collection, data):
    """ Добавляет вакансии в коллекцию """

    for doc in data:
        try:
            if doc['link'] in PRIMARY_KEYS:
                raise DuplicateKeyError('err')
            PRIMARY_KEYS.add(doc['link'])
            collection.insert_one(doc)
        except DuplicateKeyError:
            print(f'Вакансия {doc["name"]} в {doc["address"]} уже существует в базе')


def get_sample(collection, num=1):
    """ Выбирает несколько записей из коллекции и убирает поля _id (чтобы честно по ссылке проверять дубликаты)"""

    return collection.aggregate([{'$sample': {'size': num}}, {'$project': {"_id": 0}}])


def main():
    with connect_to_mongodb_collection(database_name='hw3_database',
                                       collection_name='vacancies',
                                       delete=True) as vacancies:
        list_of_vacancies = get_vacancies(text='data scientist')  # получили вакансии от парсера
        add_data_to_collection(vacancies, list_of_vacancies)  # добавили вакансии в базу
        sample = get_sample(vacancies, 4)  # получили набор из случайных вакансий
        add_data_to_collection(vacancies, sample)  # попытались добавить набор в коллекцию
        pprint_cursor_object(salary_filter(vacancies, 500000))  # фильтруем и выводим вакансии


if __name__ == '__main__':
    main()
    
#************************************

# Mongo_queries.py

# Запросы к базе монго
#  суть зпроса ограничивается одной строчкой 
#  '$match':  # Фильтрация
#               {
#                   '$or': [{'salary_min_rub': {option: value}}, {'salary_max_rub': {option: value}}]
#
# Либо 'salary_min_rub' больше введёного значения 'option: value' 
# либо 'salary_max_rub' больше введёного значения 'option: value'
# с помощтю оператора '$or' это достигается. 

def salary_filter(collection, value, option='$gte'):
    """ Переводит валюту в рубли, создаёт соответствующие поля и фильтрует результат """

    return collection.aggregate([
        {
            '$fill':  # Заполнение None
                {
                    'output':
                        {
                            'salary_min': {'value': 0},
                            'salary_max': {'value': 0},
                        }
                }
        },
        {
            '$addFields':  # Конвертация валюты
                {
                    'salary_min_rub': {'$switch': {
                        'branches': [
                            {'case': {'$eq': ['$salary_currency', 'бел.руб.']}, 'then':
                                {'$multiply': ['$salary_min', 30]}},
                            {'case': {'$eq': ['$salary_currency', 'руб.']}, 'then':
                                {'$multiply': ['$salary_min', 1]}},
                            {'case': {'$eq': ['$salary_currency', 'USD']}, 'then':
                                {'$multiply': ['$salary_min', 65]}},
                            {'case': {'$eq': ['$salary_currency', 'EUR']}, 'then':
                                {'$multiply': ['$salary_min', 70]}},
                            {'case': {'$eq': ['$salary_currency', 'сум']}, 'then':
                                {'$multiply': ['$salary_min', 0.0053]}},
                            {'case': {'$eq': ['$salary_currency', 'KZT']}, 'then':
                                {'$multiply': ['$salary_min', 0.14]}}
                        ],
                        'default': None}},
                    'salary_max_rub': {'$switch': {
                        'branches': [
                            {'case': {'$eq': ['$salary_currency', 'бел.руб.']}, 'then':
                                {'$multiply': ['$salary_max', 30]}},
                            {'case': {'$eq': ['$salary_currency', 'руб.']}, 'then':
                                {'$multiply': ['$salary_max', 1]}},
                            {'case': {'$eq': ['$salary_currency', 'USD']}, 'then':
                                {'$multiply': ['$salary_max', 65]}},
                            {'case': {'$eq': ['$salary_currency', 'EUR']}, 'then':
                                {'$multiply': ['$salary_max', 70]}},
                            {'case': {'$eq': ['$salary_currency', 'сум']}, 'then':
                                {'$multiply': ['$salary_max', 0.0053]}},
                            {'case': {'$eq': ['$salary_currency', 'KZT']}, 'then':
                                {'$multiply': ['$salary_max', 0.14]}}
                        ],
                        'default': None}}}},
        {
            '$match':  # Фильтрация
                {
                    '$or': [{'salary_min_rub': {option: value}}, {'salary_max_rub': {option: value}}]
                }
        }])
#************************************

# Parcer_hh.py

""" Немного изменённый код из 2-ой домашней работы """

from bs4 import BeautifulSoup as bs
import requests


def salary_extraction(vacancy_salary):
    salary_dict = {'min': None, 'max': None, 'cur': None}

    if vacancy_salary:
        raw_salary = vacancy_salary.getText().replace(' – ', ' ').replace(' ', '').split()
        if raw_salary[0] == 'до':
            # до 380 000 руб.
            salary_dict['max'] = int(raw_salary[1])
        elif raw_salary[0] == 'от':
            # от 50 000 руб.
            salary_dict['min'] = int(raw_salary[1])
        else:
            # 50 000 – 100 000 руб.
            salary_dict['min'] = int(raw_salary[0])
            salary_dict['max'] = int(raw_salary[1])
        salary_dict['cur'] = ''.join(raw_salary[2:])  # решение проблемы бел. руб.

    return salary_dict


def get_vacancies(text):
    main_url = 'https://spb.hh.ru/'
    page_link = '/search/vacancy'  # ссылка на первую страницу поиска

    params = {'search_field': ['name', 'company_name', 'description'], 'items_on_page': 20,
              'text': text}
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                             'Chrome/101.0.4951.67 Safari/537.36'}

    vacancies = []
    while True:
        response = requests.get(main_url+page_link,
                                params=params,
                                headers=headers)
        html = response.text
        soup = bs(html, 'html.parser')
        vacancies_soup = soup.find_all('div', {'class': ['vacancy-serp-item-body__main-info']})

        for vacancy in vacancies_soup:

            vacancy_data = {'website': 'hh.ru'}

            vacancy_title = vacancy.find('a')
            vacancy_name = vacancy_title.getText()
            vacancy_link = vacancy_title['href'][: vacancy_title['href'].index('?')]
            vacancy_salary = salary_extraction(vacancy.find('span', {'class': ['bloko-header-section-3']}))
            try:
                vacancy_employer = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}).getText().replace('\xa0', ' ')
                vacancy_address = vacancy.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).getText().replace('\xa0', ' ')
            except AttributeError:
                continue

            vacancy_data['name'] = vacancy_name
            vacancy_data['link'] = vacancy_link
            vacancy_data['salary_min'] = vacancy_salary['min']
            vacancy_data['salary_max'] = vacancy_salary['max']
            vacancy_data['salary_currency'] = vacancy_salary['cur']
            vacancy_data['employer'] = vacancy_employer
            vacancy_data['address'] = vacancy_address

            vacancies.append(vacancy_data)

        next_page = soup.find('a', {'data-qa': 'pager-next'})
        if not next_page:
            break

        page_link = next_page['href']
    return vacancies


if __name__ == '__main__':
    print(len(get_vacancies('data scientist')))

### https://github.com/LittleFox26

In [ ]:
from pymongo import MongoClient
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint

client = MongoClient('127.0.0.1', 27017)

db = client['vacDB']
# db.drop_collection(db.vacDB)
vacDB = db.vacDB

# https://hh.ru/search/vacancy?text=Data+scientist&area=1&salary=&currency_code=RUR&experience=doesNotMatter&order_by=relevance&search_period=0&items_on_page=50&no_magic=true&L_save_area=true&from=suggest_post
main_url = 'https://hh.ru'
vacancy = 'Data Scientist'
page = 0
all_vacancies = []
params = {'text': vacancy,
          'area': 2,
          'experience': 'doesNotMatter',
          'order_by': 'relevance',
          'search_period': 0,
          'items_on_page': 19,
          'page': page}
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
                         'AppleWebKit/537.36 (KHTML, like Gecko)'
                         'Chrome/98.0.4758.141 YaBrowser/22.3.4.731 Yowser/2.5 Safari/537.36'}
response = requests.get(main_url + '/search/vacancy', params=params, headers=headers)

while True:

    with open('page.html', 'w', encoding='utf-8') as f:
        f.write(response.text)

    with open('page.html', 'r', encoding='utf-8') as f:
        html = f.read()

    soup = bs(html, 'html.parser')

    vacancies = soup.find_all('div', {'class': 'vacancy-serp-item'})

    for vacancy in vacancies:

        vacancy_info = {}
        vacancy_anchor = vacancy.find('a', {'data-qa': "vacancy-serp__vacancy-title"})
        vacancy_name = vacancy_anchor.getText()
        vacancy_info['name'] = vacancy_name

        vacancy_link = vacancy_anchor['href']
        vacancy_info['link'] = vacancy_link

        vacancy_info['site'] = main_url + '/'

        vacancy_salary = vacancy.find('span', {'data-qa': "vacancy-serp__vacancy-compensation"})
        if vacancy_salary is None:
            min_salary = None
            max_salary = None
            currency = None
        else:
            vacancy_salary = vacancy_salary.getText()
            if vacancy_salary.startswith('до'):
                max_salary = int("".join([s for s in vacancy_salary.split() if s.isdigit()]))
                min_salary = None
                currency = vacancy_salary.split()[-1]

            elif vacancy_salary.startswith('от'):
                max_salary = None
                min_salary = int("".join([s for s in vacancy_salary.split() if s.isdigit()]))
                currency = vacancy_salary.split()[-1]

            else:
                max_salary = int("".join([s for s in vacancy_salary.split('–')[1] if s.isdigit()]))
                min_salary = int("".join([s for s in vacancy_salary.split('–')[0] if s.isdigit()]))
                currency = vacancy_salary.split()[-1]

        vacancy_info['max_salary'] = max_salary
        vacancy_info['min_salary'] = min_salary
        vacancy_info['currency'] = currency

        all_vacancies.append(vacancy_info)

    next_button = soup.find('a', {'data-qa': "pager-next"})
    if next_button is None:
        break
    else:
        response = requests.get(main_url + next_button['href'], headers=headers)

print(len(all_vacancies))
# pprint(all_vacancies)

# ДОБАВЛЕНИЕ ТОЛЬКО НОВЫХ ВАКАНСИЙ В ДБ:


def db_update(database, vac_list):

# ***************************************************************************
# Комментарий преподавателя по функции ниже:

# Запрос к базе данных делается безусловный это ПЛОХО 
# database.find({}) - мы всю массу которая у нас лежит мы поднимаем наверх
# в медленные тормознутые пайтоновские списки. 
# И эти списки ещё начинаем обрабатывать по нескольким значениям
# el['name'], el['max_salary'] и т.д.
# Нам наружу поднимать не надо всё, а лишь поднять нужный результат. 


    x = True
    for vac in vac_list:
        for el in database.find({}):
            if el['name'] == vac['name'] and \
               el['max_salary'] == vac['max_salary'] and \
               el['min_salary'] == vac['min_salary'] and \
               el['currency'] == vac['currency']:
                x = False
                break
            else:
                x = True
        if x:
            database.insert_one(vac)
    return database


db_update(vacDB, all_vacancies)
# for i in vacDB.find({}):
#     print(i)

# Подбор вакансий из ДБ по ЗП:


def vacancy_by_salary(database):

    try:
        salary = int(input('Insert desired salary: '))
        s = []
        for el in database.find({'$and': [
            {'$or': [{'min_salary': {'$type': 'number'}}, {'max_salary': {'$type': 'number'}}]},
            {'$or': [{'min_salary': {'$gt': salary}}, {'max_salary': {'$gt': salary}}]}
                                         ]}):
            s.append(el)
        return pprint(s)
    except ValueError:
        print('Salary should be an integer number.')


vacancy_by_salary(vacDB)

### https://github.com/Templl

In [ ]:
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError as dke

from pprint import pprint
import json
import re

#загружаем вакансии
with open('data.txt') as json_file:
    data = json.load(json_file)

#добавляем индетификатор вакансии
for vacancy in data:
    link = vacancy['link']
    id = re.split('/', link)[4]
    vacancy['_id'] = id[:8]


###
#подключаемся к базе монго
client = MongoClient('127.0.0.1', 27017)
db = client['vacncy2005']  # база данных
vacancy = db.vacancy  # название коллекции

#загружаем в базу вакансии
for i in range(len(data)):
    try:
        vacancy.insert_one(data[i])
    except dke:
        #print(f'Вакансия "{data[i]["name"]}" уже есть в базе')
        

        
        
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError as dke

from pprint import pprint

#current_course =

#подключаемся к базе монго
client = MongoClient('127.0.0.1', 27017)
db = client['vacncy2005']  # база данных
vacancy = db.vacancy  # название коллекции

# вводе переменных для поиска
salary = input('Желаемая зарплата:')
currency = input('В какой валюте зарплата:')

#salary = 220000
#currency = 'руб.'

#result = list(vacancy.find({}))
#pprint(result)

for doc in vacancy.find(
        {'мин': {'$gte': salary},
         'макс': {'$gte': salary},
         'валюта': {'$eq': currency}}
):
    if doc == None:
        print('вакансии с такими параметрами не найдены')
    else:
        pprint(doc)

result = list(vacancy.find(
                            {'мин': {'$gte': salary},
                             'макс': {'$gte': salary},
                             'валюта': {'$eq': currency}}
))

if len(result):
    pprint(result)
else:
    print('Подходящих вакансий не найдено')